In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import DBSCAN
from umap import UMAP
from sklearn.cluster import AgglomerativeClustering
from obspy import read, Stream
from typing import Any, Tuple

In [ ]:
def save_ccfs(msnoise_dir: str, dates: np.ndarray[Any, np.dtype], filter_ref: int, 
              mov_stack: int, stations: list[str], component: str = 'ZZ',
              filter_ccfs: bool = False, freq_min: float = 0.1, freq_max: float = 0.8) -> Tuple[np.ndarray, list[str]]:
    
    stack_stream = Stream()
    dates_available: list[str] = []
    
    missing: int = 0
    
    for i, day in enumerate(dates):
        stack_mseed = os.path.join(msnoise_dir, 'STACKS','{:02d}'.format(filter_ref),
                                   '{:03d}_DAYS'.format(mov_stack), component, 
                                   f'{stations[0]}_{stations[1]}', '{}.MSEED'.format(day))
        
        if os.path.isfile(stack_mseed):
            stack_stream += read(stack_mseed)
            dates_available.append(day)
        else:
            missing += 1
        
    print(f'Missing {missing} file(s)')

    if filter_ccfs is True:
        stack_stream.filter('bandpass',freqmin=freq_min, freqmax=freq_max, zerophase=True)
    
    ccfs: np.ndarray = np.array(stack_stream) 
    return ccfs, dates_available

In [2]:
def read_ccfs(ccfs_dir: str, filename: str) -> np.ndarray:
    ccf_file: str = "{}_ccfs.npy".format(filename)
    ccf_filepath = os.path.join(ccfs_dir, ccf_file)
    data: np.ndarray = np.load(ccf_filepath)
    data: np.ndarray = data/np.max(np.abs(data))
    return data

In [3]:
def plot_original(original_dir: str, filename: str, data: np.ndarray, cmap: str = "RdBu_r") -> None:
    figure_name: str = os.path.join(original_dir, "original_{}".format(filename))
    fig, ax = plt.subplots()
    img = ax.pcolorfast(data, vmin=-0.1, vmax=0.1, cmap=cmap)
    ax.set_title(filename)
    plt.colorbar(img, extend="both")
    fig.savefig(figure_name, bbox_inches='tight')
    plt.close()

In [4]:
def plot_umap(umap_dir: str, filename: str, embedding: np.ndarray, labels: np.ndarray) -> None:
    figure_name = os.path.join(umap_dir, "umap_{}".format(filename))
    fig, ax = plt.subplots()
    for label in np.unique(labels):
        mask = labels == label
        ax.scatter(embedding[mask, 0], embedding[mask, 1], s=12, label=f"{label}")
    ax.legend(ncol=2, loc="upper left", bbox_to_anchor=(1, 1))
    ax.set_aspect("equal")
    ax.set_xlabel("UMAP 1")
    ax.set_ylabel("UMAP 2")
    ax.set_title(filename)
    fig.savefig(figure_name, bbox_inches='tight')
    plt.close()

In [5]:
def plot_cluster(cluster_dir: str, filename: str, data: np.ndarray, labels: np.ndarray) -> None:
    # Explore Clusters
    # Sort the ccf matrix by cluster
    order = np.argsort(labels)
    data_sorted_by_cluster = data[order]
    
    # Also cluster within the original space
    model = AgglomerativeClustering(n_clusters=len(np.unique(labels)))
    cluster_kmeans = model.fit_predict(data)
    order_kmeans = np.argsort(cluster_kmeans)
    
    figure_name = os.path.join(cluster_dir, "cluster_{}".format(filename))
    fig, ax = plt.subplots(1, 3, figsize=(10, 4))
    ax[0].pcolorfast(data, vmin=-0.1, vmax=0.1, cmap="RdBu_r")
    ax[0].set_title("Original")
    
    # DBSCAN
    ax[1].pcolorfast(data_sorted_by_cluster, vmin=-0.1, vmax=0.1, cmap="RdBu_r")
    ax[1].set_title("Clustered in embedding space")
    for label in np.unique(labels):
        mask = labels[order] == label
        ax[1].axhline(np.where(mask)[0][0], color="black")
    
    # Kmeans
    ax[2].pcolorfast(data[order_kmeans], vmin=-0.1, vmax=0.1, cmap="RdBu_r")
    ax[2].set_title("Clustered in time domain")
    for label in np.unique(cluster_kmeans):
        mask = cluster_kmeans[order_kmeans] == label
        ax[2].axhline(np.where(mask)[0][0], color="black")
    
    fig.suptitle("{}".format(filename))
    fig.savefig(figure_name, bbox_inches='tight')
    plt.close()

In [6]:
def main():
    ccfs_dir: str = os.path.join(os.getcwd(), 'output', 'ccfs')
    filter_refs: list[int] = [1,2,3,4]
    mov_stacks: list[int] = [1,2,5,10]
    stations: list[str] = ['VG_BANG', 'VG_LEKR']
    
    figures_dir: str = os.path.join(os.getcwd(), 'figures')
    os.makedirs(figures_dir, exist_ok=True)
    
    original_dir: str = os.path.join(figures_dir, 'original')
    os.makedirs(original_dir, exist_ok=True)
    
    umap_dir: str = os.path.join(figures_dir, 'umap')
    os.makedirs(umap_dir, exist_ok=True)
    
    cluster_dir: str = os.path.join(figures_dir, 'cluster')
    os.makedirs(cluster_dir, exist_ok=True)
    
    for filter_ref in filter_refs:
        for mov_stack in mov_stacks:
            filename: str = "{}_{}_{:02d}_{:03d}_DAYS".format(stations[0], stations[1], filter_ref, mov_stack)
            data: np.ndarray = read_ccfs(ccfs_dir=ccfs_dir, filename=filename)
            
            # Plotting original
            plot_original(original_dir, filename, data)
            
            # Compress with UMAP and get embedding data
            model_umap = UMAP(n_components=2, metric="correlation", n_neighbors=20, min_dist=1)
            embedding_data = model_umap.fit_transform(data)
            
            # Clustering DBSCAN (Density-Based Spatial Clustering of Applications with Noise)
            # Cluster labels. Noisy samples are given the label -1
            model = DBSCAN(eps=1)
            labels: np.ndarray = model.fit_predict(embedding_data)
            
            # Plotting UMAP clustering
            plot_umap(umap_dir, filename, embedding_data, labels)
            
            # Plotting
            plot_cluster(cluster_dir, filename, data, labels)

In [7]:
main()